### Seq2Seq model for English to Chinese translation (baseline model)

In [38]:
import json
import numpy as np
import pickle
from collections import Counter
import string
import re 

### Choosing, loading, and cleaning dataset

- going to use datasets from: https://www.kaggle.com/datasets/qianhuan/translation?resource=download

In [ ]:
train_set_path = "dataset/translation2019zh_train.json"

train_set = []
with open(train_set_path) as f:
    for line in f:
        train_set.append(json.loads(line))

print(len(train_set))
print(train_set[0])

5161434
{'english': 'For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.', 'chinese': '为了更好的锐度，但是附带的会多一些颗粒度，可以使用这个显影剂的1：1稀释液。'}


We want to lower the size of this dataset, for testing purposes.
- right now 5.1M sentences
- let's make it 10,000 sentences...

In [12]:
# get 10,000 random indices 
sampled_indices = np.random.choice(len(train_set), 10000)

train_subset = [train_set[i] for i in sampled_indices]
print(train_subset[0])
with open('dataset/train_set_mini.pkl', 'wb') as f:
    pickle.dump(train_subset, f)

{'english': 'The amphibian & reptile hall, the marine animal hall, the tropic monkey hall, the bird garden are to be built on the flat ground, and the Garden will welcome Chinese and foreign guests with new looks.', 'chinese': '两栖——爬行动物馆、海兽馆、热带猴馆、鸟园等新型建筑将平地而起，以新的容貌迎接中外宾客。'}


In [5]:
with open('dataset/train_set_mini.pkl', 'rb') as f:
    train_set_mini = pickle.load(f)

print(train_set_mini[0])

{'english': 'The amphibian & reptile hall, the marine animal hall, the tropic monkey hall, the bird garden are to be built on the flat ground, and the Garden will welcome Chinese and foreign guests with new looks.', 'chinese': '两栖——爬行动物馆、海兽馆、热带猴馆、鸟园等新型建筑将平地而起，以新的容貌迎接中外宾客。'}


Train set made. 
Now to work on the actual architecture

## Preprocessing steps:
- we want to maintain vocabulary for english and chinese. 
    - this is simple, just use a counter() and then limit it to if it appears within 5

- In terms of encoding, we want to use sequence input, so a sentence becomes a list [3, 100, 8, 9], where each number corresponds to the index of the word in the dictionary
    - we want to do this because LSTMs keep track of the word relationships at the sentence level
    - then use nn.Embedding?
        - nn.Embedding allows us to create a matrix representing the vocabulary. It allows us to train some nuance into the words, where instead of typical BoW where each word is just an index, each word is now a vector, which allows us to train some meaning into the word
        - the embedding is a matrix size (vocab length, dim). Length vocab length because each row in the matrix corresponds to a word in the vocab, ( row of index = index of word in vocab :) )
        

# now to work on the vocabulary

In [47]:
## helper functions 
def remove_punctuation(text):
    '''
    Get rid of all punctuation from string text
    '''
    return text.translate(str.maketrans('', '', string.punctuation))

def get_words_from_sentence(s):
    '''
    Gets words from sentence 
    '''
    return s.split(' ')

def clean_en_pair(pair):
    '''
    Cleans the english from the pair 
    '''
    return get_words_from_sentence(remove_punctuation(pair['english']).lower())

def get_en_vocab(train_set):
    '''
    get_en_dict:
        Gets an english vocab from train_set as a dict 
    '''
    # get only the english sentences, list of strings 
    en_sentences = [clean_en_pair(pair) for pair in train_set]
    en_sentences_flattened = [word for sentence in en_sentences for word in sentence]
    print(f"Words pre-clean {len(en_sentences_flattened)}")
    en_sentences_flattened = [word for word in en_sentences_flattened if word != '']
    print(f"Words post-clean {len(en_sentences_flattened)}")
    
    word_counts = Counter(en_sentences_flattened)
    # with word counts, now we limit the vocabulary to words that happen at least 5 times
    en_vocab = {}
    # {word: index}
    idx = 0
    for word in ["<SOS>", "<EOS>", "<UNK>"]:
        en_vocab[word] = idx 
        idx += 1
    for word, occurrences in word_counts.items():
        if occurrences >= 5:
            en_vocab[word] = idx 
            idx += 1
    return en_vocab

def remove_zh_punctuation(text):
    cleaned = re.sub(r'[，。！？【】（）《》“”‘’、]', '', text)
    cleaned = re.sub(r'\s+', '', cleaned)
    return cleaned

def get_zh_vocab(train_set):
    '''
    get_zh_vocab:
        Gets an zh vocab from train_set as a dict 
    '''
    zh_sentences = [list(remove_zh_punctuation(pair['chinese'])) for pair in train_set]
    zh_sentences_flattened = [word for sentence in zh_sentences for word in sentence]
    print(len(zh_sentences_flattened))

    word_counts = Counter(zh_sentences_flattened)
    zh_vocab = {}

    idx = 0 
    for word in ["<SOS>", "<EOS>", "<UNK>"]:
        zh_vocab[word] = idx 
        idx += 1 
    for word, occurrences in word_counts.items():
        if occurrences >= 5: 
            zh_vocab[word] = idx 
            idx += 1 
    return zh_vocab

en_vocab = get_en_vocab(train_set_mini)
print(en_vocab)

zh_vocab = get_zh_vocab(train_set_mini)
print(zh_vocab)

Words pre-clean 192167
Words post-clean 191139
{'<SOS>': 0, '<EOS>': 1, '<UNK>': 2, 'the': 3, 'hall': 4, 'marine': 5, 'animal': 6, 'bird': 7, 'garden': 8, 'are': 9, 'to': 10, 'be': 11, 'built': 12, 'on': 13, 'flat': 14, 'ground': 15, 'and': 16, 'will': 17, 'welcome': 18, 'chinese': 19, 'foreign': 20, 'guests': 21, 'with': 22, 'new': 23, 'looks': 24, 'its': 25, 'important': 26, 'show': 27, 'when': 28, 'you': 29, 'dont': 30, 'play': 31, 'well': 32, 'were': 33, 'still': 34, 'fighting': 35, 'for': 36, 'club': 37, 'get': 38, 'right': 39, 'result': 40, 'at': 41, 'same': 42, 'time': 43, 'i': 44, 'authorized': 45, 'company': 46, 'in': 47, 'guangxi': 48, 'a': 49, 'survey': 50, 'of': 51, 'all': 52, 'circumstances': 53, 'am': 54, 'his': 55, 'wife': 56, 'is': 57, 'trying': 58, 'pull': 59, 'him': 60, 'some': 61, 'specific': 62, 'conditions': 63, 'had': 64, 'vibration': 65, 'few': 66, 'ones': 67, 'hydraulic': 68, 'instability': 69, 'after': 70, 'successful': 71, 'sir': 72, 'went': 73, 'explore': 74,

In [51]:
with open('vocab/en_vocab.pkl', 'wb') as f:
    pickle.dump(en_vocab, f)

with open('vocab/zh_vocab.pkl', 'wb') as f:
    pickle.dump(zh_vocab, f)

In [53]:
with open('vocab/en_vocab.pkl', 'rb') as f:
    en_vocab = pickle.load(f)

with open('vocab/zh_vocab.pkl', 'rb') as f:
    zh_vocab = pickle.load(f)
print(en_vocab)
print(zh_vocab)

{'<SOS>': 0, '<EOS>': 1, '<UNK>': 2, 'the': 3, 'hall': 4, 'marine': 5, 'animal': 6, 'bird': 7, 'garden': 8, 'are': 9, 'to': 10, 'be': 11, 'built': 12, 'on': 13, 'flat': 14, 'ground': 15, 'and': 16, 'will': 17, 'welcome': 18, 'chinese': 19, 'foreign': 20, 'guests': 21, 'with': 22, 'new': 23, 'looks': 24, 'its': 25, 'important': 26, 'show': 27, 'when': 28, 'you': 29, 'dont': 30, 'play': 31, 'well': 32, 'were': 33, 'still': 34, 'fighting': 35, 'for': 36, 'club': 37, 'get': 38, 'right': 39, 'result': 40, 'at': 41, 'same': 42, 'time': 43, 'i': 44, 'authorized': 45, 'company': 46, 'in': 47, 'guangxi': 48, 'a': 49, 'survey': 50, 'of': 51, 'all': 52, 'circumstances': 53, 'am': 54, 'his': 55, 'wife': 56, 'is': 57, 'trying': 58, 'pull': 59, 'him': 60, 'some': 61, 'specific': 62, 'conditions': 63, 'had': 64, 'vibration': 65, 'few': 66, 'ones': 67, 'hydraulic': 68, 'instability': 69, 'after': 70, 'successful': 71, 'sir': 72, 'went': 73, 'explore': 74, 'south': 75, 'pole': 76, 'china': 77, 'nationa